In [ ]:
# Using Python

In [10]:
import pandas as pd
import sqlite3

# Load the Excel data into a Pandas DataFrame
excel_file_path = "C:/Users/daksh/Downloads/Time Series.xlsx"
df = pd.read_excel(excel_file_path)

# Convert 'Start' and 'End' columns to datetime
df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])

# Establish a connection to the SQLite database
db_file_path = "bot_activities.db"
conn = sqlite3.connect(db_file_path)

# Insert data from the DataFrame into the database table
df.to_sql('bot_activities', conn, if_exists='replace', index=False)

# Commit changes and close the connection
conn.commit()
conn.close()

print("Data has been loaded and stored in the SQLite database.")


Data has been loaded and stored in the SQLite database.


In [17]:
# Using SQL
%load_ext sql
%sql sqlite:///ecommerce.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [24]:
%%sql
PRAGMA table_info(bot_activities);

 * sqlite:///ecommerce.db
Done.


cid,name,type,notnull,dflt_value,pk
0,id,INT,0,None,0
1,bot,TEXT,0,None,0
2,start,TEXT,0,None,0
3,end,TEXT,0,None,0
4,activities,TEXT,0,None,0
5,session_id,INTEGER,0,None,0


In [30]:
%%sql
WITH RankedActivities AS (
    SELECT
        bot,
        start,
        end,
        activities,
        ROW_NUMBER() OVER (PARTITION BY bot ORDER BY start) AS rn
    FROM
        bot_activities
),
GroupedActivities AS (
    SELECT
        ra1.bot,
        ra1.start AS PeriodStart,
        ra1.end AS PeriodEnd,
        GROUP_CONCAT(ra1.activities, '\n') AS AggregatedActivities
    FROM
        RankedActivities ra1
    WHERE
        NOT EXISTS (
            SELECT 1
            FROM RankedActivities ra2
            WHERE ra2.bot = ra1.bot AND ra2.start = ra1.end
        )
    GROUP BY
        ra1.bot, ra1.start, ra1.end
)
SELECT
    bot,
    PeriodStart,
    PeriodEnd,
    AggregatedActivities
FROM
    GroupedActivities
ORDER BY
    bot, PeriodStart;


 * sqlite:///ecommerce.db
Done.


bot,PeriodStart,PeriodEnd,AggregatedActivities
